# Week 4 Examples — Minimal JSON-RPC, Timeouts, Retries, Idempotency

This notebook implements a tiny JSON-RPC over TCP with:
- Request IDs for correlation and deduplication
- Client-side timeouts and exponential backoff with jitter
- Idempotent handler on the server (increment with idempotency key)
- Injected delays to observe retries and tail latency

Requires Python 3.9+.

---

## Protocol sketch
- Client sends JSON object framed by 4-byte big-endian length:
  `{ "id": "uuid", "method": "inc", "params": {"key": "k", "delta": 1} }`
- Server replies with either `{ "id": "uuid", "result": ... }` or `{ "id": "uuid", "error": {"code":..., "message":...} }`
- Idempotency: server caches results by `(method, id)` for a TTL window and replays result on duplicate.


In [ ]:
import socket, struct, threading, time, json, random, uuid
from contextlib import closing


def send_frame(sock: socket.socket, data: bytes):
    sock.sendall(struct.pack('!I', len(data)) + data)


def recv_all(sock: socket.socket, n: int) -> bytes:
    buf = bytearray()
    while len(buf) < n:
        chunk = sock.recv(n - len(buf))
        if not chunk:
            raise ConnectionError('socket closed')
        buf.extend(chunk)
    return bytes(buf)


def recv_frame(sock: socket.socket) -> bytes:
    (length,) = struct.unpack('!I', recv_all(sock, 4))
    return recv_all(sock, length)


## JSON-RPC Server with idempotent increment
- Methods: `inc` increments a counter by `delta` for a given `key`
- Idempotency: cache responses by request `id` for a TTL
- Failure injection: optional artificial delay and drop probability


In [ ]:
class JsonRpcServer:
    def __init__(self, host='127.0.0.1', port=50777, drop_p=0.0, delay_ms=0):
        self.addr = (host, port)
        self.sock = None
        self.running = threading.Event()
        self.state = {}  # counters by key
        self.cache = {}  # id -> (deadline, response_bytes)
        self.cache_ttl = 10.0
        self.drop_p = drop_p
        self.delay_ms = delay_ms

    def start(self):
        self.running.set()
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind(self.addr)
        self.sock.listen(64)
        threading.Thread(target=self._accept_loop, daemon=True).start()
        print('Server listening on', self.addr)

    def stop(self):
        self.running.clear()
        try:
            self.sock.close()
        except Exception:
            pass

    def _accept_loop(self):
        while self.running.is_set():
            try:
                c, _ = self.sock.accept()
            except OSError:
                break
            threading.Thread(target=self._client_loop, args=(c,), daemon=True).start()

    def _client_loop(self, c: socket.socket):
        try:
            while self.running.is_set():
                frame = recv_frame(c)
                if random.random() < self.drop_p:
                    # Drop on floor to simulate loss (client should retry)
                    continue
                if self.delay_ms:
                    time.sleep(self.delay_ms/1000.0)
                req = json.loads(frame.decode('utf-8'))
                rid = req.get('id')
                now = time.time()
                # Serve from idem cache if present
                data = self.cache.get(rid)
                if data and data[0] > now:
                    send_frame(c, data[1])
                    continue
                resp = self._handle(req)
                resp_bytes = json.dumps(resp).encode('utf-8')
                self.cache[rid] = (now + self.cache_ttl, resp_bytes)
                send_frame(c, resp_bytes)
        except Exception:
            pass
        finally:
            try: c.close()
            except Exception: pass

    def _handle(self, req: dict) -> dict:
        rid = req.get('id')
        method = req.get('method')
        params = req.get('params') or {}
        try:
            if method == 'inc':
                key = params['key']
                delta = int(params.get('delta', 1))
                self.state[key] = self.state.get(key, 0) + delta
                return {"id": rid, "result": {"key": key, "value": self.state[key]}}
            else:
                return {"id": rid, "error": {"code": -32601, "message": "Method not found"}}
        except Exception as e:
            return {"id": rid, "error": {"code": -32000, "message": str(e)}}


## JSON-RPC Client with retries and backoff
- Per-RPC timeout
- Exponential backoff with jitter
- Deduplicate responses by `id`


In [ ]:
class JsonRpcClient:
    def __init__(self, addr=('127.0.0.1', 50777), timeout=0.3, max_retries=3,
                 backoff_base=0.05, backoff_factor=2.0, jitter=0.1):
        self.addr = addr
        self.timeout = timeout
        self.max_retries = max_retries
        self.backoff_base = backoff_base
        self.backoff_factor = backoff_factor
        self.jitter = jitter

    def call(self, method: str, params: dict):
        rid = str(uuid.uuid4())
        payload = json.dumps({"id": rid, "method": method, "params": params}).encode('utf-8')
        attempt = 0
        deadline = time.time() + max(self.timeout, 0.05) * (self.max_retries+1)
        while attempt <= self.max_retries and time.time() < deadline:
            attempt += 1
            try:
                with closing(socket.create_connection(self.addr, timeout=self.timeout)) as s:
                    s.settimeout(self.timeout)
                    send_frame(s, payload)
                    resp = json.loads(recv_frame(s).decode('utf-8'))
                    if resp.get('id') == rid:
                        return resp
            except Exception:
                # Backoff
                delay = self.backoff_base * (self.backoff_factor ** (attempt-1))
                delay *= (1.0 + self.jitter * random.random())
                time.sleep(delay)
        raise TimeoutError(f'RPC {method} timed out after {attempt} attempts')


## Demo: observe retries and idempotency under delay and drop
1) Start server with delay and drop.
2) Run multiple `inc` calls with same `id` via forced reuse to show idempotent result.
3) Observe tail latencies and retry counts.


In [ ]:
server = JsonRpcServer(delay_ms=120, drop_p=0.2)
server.start()

client = JsonRpcClient(timeout=0.1, max_retries=5, backoff_base=0.03)

# Normal unique calls
for i in range(5):
    resp = client.call('inc', {"key": "counter", "delta": 1})
    print('resp', resp)

# Forced duplicate by reusing a known id to simulate client retry with same id
rid = str(uuid.uuid4())
payload = json.dumps({"id": rid, "method": "inc", "params": {"key": "counter", "delta": 5}}).encode('utf-8')
with closing(socket.create_connection(server.addr, timeout=1)) as s:
    send_frame(s, payload)
    _ = recv_frame(s)
# Send duplicate
with closing(socket.create_connection(server.addr, timeout=1)) as s:
    send_frame(s, payload)
    dup = json.loads(recv_frame(s).decode('utf-8'))
    print('duplicate resp (same id):', dup)

server.stop()


## Exercises
1) Change client backoff parameters; measure impact on success rate and latency under drop_p=0.3.\
2) Add a new method `get` that returns the current value without changes; ensure it’s naturally idempotent.\
3) Add a bounded queue on the server to apply backpressure and return an error when overloaded.
